## PREPROCESAMIENTO DE DATOS - MINERÍA DE DATOS
**Estudiante:** Nadiabeth Diana Mallqui Apaza    
**Código:** 171063

In [56]:
# Importamos la librería "findspark"
import findspark
# "findspark.init()" permite que pyspark se pueda importar como una biblioteca regular.
findspark.init()

In [57]:
# Importamos las librerías necesarias
import math
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
spark = SparkSession(sc)
sc =SparkContext.getOrCreate()

### 1° Algoritmo : Bag of Words

In [58]:
# Módulo BoW contabiliza las veces que aparece una palabra
def BoW(data, columns):
    # Convertimos nuestros datos de entrada en dataframe
    df = spark.createDataFrame(data, columns)
    # Mostramos
    df.show()
    # Realizamos el proceso del bag of words
    bow = df.rdd\
        .filter(lambda x: x.Document)\
        .map( lambda x: x.Document.replace(',',' ').replace('.',' ').replace('-',' ').lower())\
        .flatMap(lambda x: x.split())\
        .map(lambda x: (x, 1))
    # Mostramos las palabras y la cantidad de veces que se repiten
    print("Bag of Words")
    print(bow.reduceByKey(lambda x,y:x+y).take(10))

In [59]:
# Ejemplo Bag of Words
data = [['Jan', 'John', 'This is a document'],
        ['Feb', 'Mary', 'A book by Mary'],
        ['Mar', 'Luke', 'Newspaper article'],
        ['Apr', 'Mark', None]]
columns = ['Month', 'Author', 'Document']
# Hacemos uso del módulo BoW
BoW(data,columns)

+-----+------+------------------+
|Month|Author|          Document|
+-----+------+------------------+
|  Jan|  John|This is a document|
|  Feb|  Mary|    A book by Mary|
|  Mar|  Luke| Newspaper article|
|  Apr|  Mark|              null|
+-----+------+------------------+

Bag of Words
[('book', 1), ('mary', 1), ('a', 2), ('by', 1), ('article', 1), ('this', 1), ('is', 1), ('newspaper', 1), ('document', 1)]


### 2° Algoritmo : Escalonamiento

In [60]:
# Librerías necesarias para el escalonamiento 
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors

# Data de entrada
dataFrame = spark.createDataFrame([
    (0, Vectors.dense([1.0, 0.1, -1.0]),),
    (1, Vectors.dense([2.0, 1.1, 1.0]),),
    (2, Vectors.dense([3.0, 10.1, 3.0]),)
], ["id", "features"])

# Ingresamos parámetros para el MinMaxScaler
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

# Calcule estadísticas resumidas y genere MinMaxScalerModel
scalerModel = scaler.fit(dataFrame)

# Cambiar la escala de cada característica al rango [mínimo, máximo]
scaledData = scalerModel.transform(dataFrame)
print("Funciones escaladas al rango: [%f, %f]" % (scaler.getMin(), scaler.getMax()))
scaledData.select("features", "scaledFeatures").show()

Funciones escaladas al rango: [0.000000, 1.000000]
+--------------+--------------+
|      features|scaledFeatures|
+--------------+--------------+
|[1.0,0.1,-1.0]|     (3,[],[])|
| [2.0,1.1,1.0]| [0.5,0.1,0.5]|
|[3.0,10.1,3.0]| [1.0,1.0,1.0]|
+--------------+--------------+



### 3° Algoritmo : Binario

In [61]:
# Librería Binarizer
from pyspark.ml.feature import Binarizer

# DataFrame de entrada
continuousDataFrame = spark.createDataFrame([
    (0, 0.1),
    (1, 0.8),
    (2, 0.2)
], ["id", "feature"])

# Método Binarizer
binarizer = Binarizer(threshold=0.5, inputCol="feature", outputCol="binarized_feature")

# Mostramos en dataframe
binarizedDataFrame = binarizer.transform(continuousDataFrame)

print("Salida del binarizador con umbral = %f" % binarizer.getThreshold())
binarizedDataFrame.show()

Salida del binarizador con umbral = 0.500000
+---+-------+-----------------+
| id|feature|binarized_feature|
+---+-------+-----------------+
|  0|    0.1|              0.0|
|  1|    0.8|              1.0|
|  2|    0.2|              0.0|
+---+-------+-----------------+



### 4° Algoritmo : Distancia de Jaccard

In [62]:
# Importamos la librerías a utilizar
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col

# Ingresamos los datos A como vectores
dataA = [(0, Vectors.sparse(6, [0, 1, 2], [1.0, 1.0, 1.0]),),
         (1, Vectors.sparse(6, [2, 3, 4], [1.0, 1.0, 1.0]),),
         (2, Vectors.sparse(6, [0, 2, 4], [1.0, 1.0, 1.0]),)]
# Creamos un DataFrame con los datos anteriormente ingresados (datos A)
dfA = spark.createDataFrame(dataA, ["id", "features"])

# De igual manera, ingresamos los datos B como vectores
dataB = [(3, Vectors.sparse(6, [1, 3, 5], [1.0, 1.0, 1.0]),),
         (4, Vectors.sparse(6, [2, 3, 5], [1.0, 1.0, 1.0]),),
         (5, Vectors.sparse(6, [1, 2, 4], [1.0, 1.0, 1.0]),)]
# Creamos un DataFrame con los datos anteriormente ingresados (datos B)
dfB = spark.createDataFrame(dataB, ["id", "features"])

key = Vectors.sparse(6, [1, 3], [1.0, 1.0])

# Clase pública MinHashLSH para distancia de Jaccard.
# inputCol() ----> Parámetro para el nombre de la columna de entrada.
# outputCol() ----> Parámetro para el nombre de la columna de salida.
# numHashTables() ----> Parámetro para el número de tablas hash utilizadas en la amplificación OR de LSH.

mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=5)
model = mh.fit(dfA)

# Transformación de características
print("Conjunto de datos donde los valores hash se almacenan en la columna 'hashes'")
model.transform(dfA).show()

# Realizamos un cálculo aproximado de unión de similitud - Distancia de Jaccard 
print("Mostramos la Distancia de Jaccard menor a 0.6")
model.approxSimilarityJoin(dfA, dfB, 0.6, distCol="Distancia de Jaccard")\
    .select(col("datasetA.id").alias("idA"),
            col("datasetB.id").alias("idB"),
            col("Distancia de Jaccard")).show()


Conjunto de datos donde los valores hash se almacenan en la columna 'hashes'
+---+--------------------+--------------------+
| id|            features|              hashes|
+---+--------------------+--------------------+
|  0|(6,[0,1,2],[1.0,1...|[[2.60304514E8], ...|
|  1|(6,[2,3,4],[1.0,1...|[[4.68158287E8], ...|
|  2|(6,[0,2,4],[1.0,1...|[[2.60304514E8], ...|
+---+--------------------+--------------------+

Mostramos la Distancia de Jaccard menor a 0.6
+---+---+--------------------+
|idA|idB|Distancia de Jaccard|
+---+---+--------------------+
|  2|  5|                 0.5|
|  1|  5|                 0.5|
|  0|  5|                 0.5|
+---+---+--------------------+



### 5° Algoritmo : N-Grams

In [63]:
# Importamos la librerías a utilizar
from pyspark.ml.feature import NGram
from pyspark.sql import SparkSession

# Creamos un DataFrame con los datos ingresados
wordDataFrame = spark.createDataFrame([
        (0, ["Hola", "yo", "escuché", "sobre", "Spark"]),
        (1, ["Ojalá", "Java", "pudiera", "usar", "clases", "de", "casos"]),
        (2, ["Modelos", "Regresión", "Logística", "son", "ordenados"])
    ], ["id", "Palabras"])

# Mostramos el DataFRame con las palabras ingresadas
wordDataFrame.show(truncate=False)

# EJEMPLOS
# Calculamos los n-gramas que deseamos
print("Mostramos los n-gramas con n=2")
ngram1 = NGram(n=2, inputCol="Palabras", outputCol="N-grams")
# Transformamos las palabras en un dataframe
ngramDataFrame = ngram1.transform(wordDataFrame)
# Mostramos el dataframe con los n-gramas respectivos
ngramDataFrame.select("N-grams").show(truncate=False)
#-----------------------------------------------------------------
# Calculamos los n-gramas que deseamos
print("Mostramos los n-gramas con n=3")
ngram2 = NGram(n=3, inputCol="Palabras", outputCol="N-grams")
# Transformamos las palabras en un dataframe
ngramDataFrame = ngram2.transform(wordDataFrame)
# Mostramos el dataframe con los n-gramas respectivos
ngramDataFrame.select("N-grams").show(truncate=False)

+---+-----------------------------------------------+
|id |Palabras                                       |
+---+-----------------------------------------------+
|0  |[Hola, yo, escuché, sobre, Spark]              |
|1  |[Ojalá, Java, pudiera, usar, clases, de, casos]|
|2  |[Modelos, Regresión, Logística, son, ordenados]|
+---+-----------------------------------------------+

Mostramos los n-gramas con n=2
+--------------------------------------------------------------------------+
|N-grams                                                                   |
+--------------------------------------------------------------------------+
|[Hola yo, yo escuché, escuché sobre, sobre Spark]                         |
|[Ojalá Java, Java pudiera, pudiera usar, usar clases, clases de, de casos]|
|[Modelos Regresión, Regresión Logística, Logística son, son ordenados]    |
+--------------------------------------------------------------------------+

Mostramos los n-gramas con n=3
+------------------